In [1]:
import os
os.getcwd()

'c:\\Users\\dshem\\OneDrive\\Desktop\\projects\\project-2\\IPP-1'

In [24]:
from sklearn import preprocessing
from ipp.exception import InsuranceException
from ipp.logger import logging
from ipp.entity.config_entity import DataTransformationConfig 
from ipp.entity.artifact_entity import DataIngestionArtifact,DataValidationArtifact,DataTransformationArtifact
import sys,os
import numpy as np
from ipp.constant import *
from ipp.util.util import read_yaml_file,save_object,save_numpy_array_data,load_data

#from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder


class DataTransformation:
    def __init__(self,
               data_transformation_config: DataTransformationConfig,
               data_ingestion_artifact: DataIngestionArtifact,
               data_validation_artifact: DataValidationArtifact):
        try:
            logging.info(f"{'**'*20}data transformation log started ! {'**'*20}")
            self.data_transformation_config=data_transformation_config
            self.data_ingestion_artifact=data_ingestion_artifact
            self.data_validation_artifact=data_validation_artifact

        except Exception as e:
            raise InsuranceException(e,sys) from e
        
    def get_data_transformer_obj(self)->ColumnTransformer:
        try:
            schema_file_path=self.data_validation_artifact.schema_file_path
            #print(schema_file_path[0])
            dataset_schema=read_yaml_file(file_path=schema_file_path)
            #print(dataset_schema)

            num_columns=dataset_schema[NUMERICAL_COLUMN_KEY]
            cat_columns=dataset_schema[CATEGORICAL_COLUMN_KEY]
            #print(num_columns,cat_columns)
            num_pipeline=Pipeline(steps=[("imputer",SimpleImputer(strategy='median')),
                                        ("scaler",StandardScaler())
                                        ])
            
            cat_pipeline=Pipeline(steps=[("imputer",SimpleImputer(strategy='most_frequent')),
                                         ("encoder",OneHotEncoder()),
                                         ("scaler",StandardScaler(with_mean=False))
                                        ])
            
            preprocessing=ColumnTransformer([("num_pipelene",num_pipeline,num_columns),
                                             ("cat_pipeline",cat_pipeline,cat_columns),
                                            ])
            return preprocessing
        except Exception as e:
            raise InsuranceException(e,sys) from e
    
data_trans=DataTransformation(data_ingestion_artifact=DataIngestionArtifact,data_transformation_config=DataTransformationConfig,data_validation_artifact=DataValidationArtifact)

In [27]:
from ipp.entity.artifact_entity import DataValidationArtifact,DataIngestionArtifact
from ipp.util.util import read_yaml_file,load_data
from ipp.constant import *
data_ing=DataIngestionArtifact(train_file_path='C:\\Users\\dshem\\OneDrive\\Desktop\\projects\\project-2\\IPP-1\\Insurance\\artifact\\data_ingestion\\2024-03-02-09-27-20\\ingested_data\\train\\insurance.csv', test_file_path='C:\\Users\\dshem\\OneDrive\\Desktop\\projects\\project-2\\IPP-1\\Insurance\\artifact\\data_ingestion\\2024-03-02-09-27-20\\ingested_data\\test\\insurance.csv', is_ingested=True, message='Data ingestion completed successfully.')
data_val=DataValidationArtifact(schema_file_path='C:\\Users\\dshem\\OneDrive\\Desktop\\projects\\project-2\\IPP-1\\config\\schema.yaml', report_file_path=None, report_page_file_path=None, is_validated=True, message='data validation performed successfully!')
schema_file_path =data_val.schema_file_path
train_file_path=data_ing.train_file_path
schema = read_yaml_file(file_path=schema_file_path)
print(schema)
train_df = load_data(file_path=train_file_path, schema_file_path=schema_file_path)
target_column_name = schema[TARGET_COLUMN_KEY]
#train_df

input_feature_train_df = train_df.drop(columns=[target_column_name],axis=1)
#print(input_feature_train_df)
preprocessing_obj = data_trans.get_data_transformer_obj()

input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)

{'columns': {'age': 'int', 'sex': 'category', 'bmi': 'float', 'children': 'int', 'smoker': 'category', 'region': 'category', 'expenses': 'float'}, 'numerical_columns': ['age', 'bmi', 'children', 'expenses'], 'categorical_columns': ['sex', 'smoker', 'region'], 'target_column': 'expenses', 'domain_value': {'region': ['southeast', 'southwest', 'northeast', 'northwest']}}


InsuranceException: 
        Error occured in script: 
        [ C:\Users\dshem\AppData\Local\Temp\ipykernel_19432\2185755444.py ] at 
        try block line number: [36] and exception block line number: [56] 
        error message: [
        Error occured in script: 
        [ c:\Users\dshem\OneDrive\Desktop\projects\project-2\IPP-1\ipp\util\util.py ] at 
        try block line number: [15] and exception block line number: [18] 
        error message: [expected str, bytes or os.PathLike object, not _tuplegetter]
        ]
        